In [2]:
%load_ext autoreload
%autoreload 2
import geopandas as gpd
import numpy as np
import pandas as pd
import os
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor

# local import
from make_datasets import make_data
import models

2023-07-17 15:14:12.740835: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:

data_dir = '/Users/jyontika/Desktop/Python/GitHub/opioid-overdose-models/jyontika-MA-data/data/'
annual_path = os.path.join(data_dir, 'clean_annual_tract')
data_gdf = gpd.read_file(annual_path)

,geoid,year,deaths,month,STATEFP,COUNTYFP,TRACTCE,NAME,NAMELSAD,MTFCC,...,month_sinc,season,season_sin,quarter,qtr_since_,year_since,self_t-1,neighbor_t,timestep,geometry
0,25001010100,2000,0.0,1,25,001,10100,101,Census Tract 101,G5020,...,0.0,jan-jun,0.0,1.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((-70.25001 42.06410, -70.24959 42.065..."
1,25001010100,2001,5.0,1,25,001,10100,101,Census Tract 101,G5020,...,12.0,jan-jun,2.0,1.0,4.0,1.0,0.0,0.0,1.0,"POLYGON ((-70.25001 42.06410, -70.24959 42.065..."
2,25001010100,2002,0.0,1,25,001,10100,101,Census Tract 101,G5020,...,24.0,jan-jun,4.0,1.0,8.0,2.0,0.0,0.0,2.0,"POLYGON ((-70.25001 42.06410, -70.24959 42.065..."
3,25001010100,2003,1.0,1,25,001,10100,101,Census Tract 101,G5020,...,36.0,jan-jun,6.0,1.0,12.0,3.0,0.0,0.0,3.0,"POLYGON ((-70.25001 42.06410, -70.24959 42.065..."
4,25001010100,2004,0.0,1,25,001,10100,101,Census Tract 101,G5020,...,48.0,jan-jun,8.0,1.0,16.0,4.0,0.0,0.0,4.0,"POLYGON ((-70.25001 42.06410, -70.24959 42.065..."


Process dataframe into a data frame with a Multiindex on location and time

In [4]:

# Name the important columns
timestep_col = 'timestep'
geography_col = 'geoid'
outcome_col = 'deaths'

# These are the columns we could possibly want in the X dataframe
x_idx_cols = [geography_col, 'lat', 'lon', timestep_col,
              'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
              'svi_pctile', 'year',
              'neighbor_t', 'deaths']

# These are the columns we could want in the Y dataframe
y_idx_cols = [geography_col, timestep_col, outcome_col]

# These are the features we want
features_only = ['deaths']
add_spacetime = True
add_svi = True
if add_spacetime:
    features_only += ['lat', 'lon', timestep_col]
if add_svi:
    features_only += ['theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc', 'svi_pctile']


validation_year = 2018
first_test_year = 2019
last_test_year = 2020
first_test_timestep = 19
last_test_timestep = 20
lookback_years=5
first_train_eval_year = validation_year - lookback_years
last_train_eval_year = validation_year -1

In [5]:
# Create the multiindex
multiindexed_gdf = data_gdf.set_index([geography_col, timestep_col])

# re-add the timestep column as a feature because it's useful
multiindexed_gdf[timestep_col] = multiindexed_gdf.index.get_level_values(timestep_col)

# Track number of locations
num_geoids = len(data_gdf[geography_col].unique())

In [6]:
x_BSF, y_BS = make_data(multiindexed_gdf, first_train_eval_year, last_train_eval_year, lookback_years,
          features_only, num_geoids)
x_test_BSF, y_test_BS =make_data(multiindexed_gdf, first_test_year, last_test_year, lookback_years,
          features_only, num_geoids)

# For the weighted historical average model, we only use deaths as features
x_BSF_death_only, y_BS_death_only = make_data(multiindexed_gdf, first_train_eval_year, last_train_eval_year, lookback_years,
          ['deaths'], num_geoids)
x_test_BSF_death_only, y_test_BS_death_only =make_data(multiindexed_gdf, first_test_year, last_test_year, lookback_years,
          ['deaths'], num_geoids)

In [7]:
x_BSF.shape

TensorShape([5, 1620, 45])

In [8]:
y_BS.shape

TensorShape([5, 1620])

In [13]:
bpr_over_time_zeroes = models.all_zeroes_model(multiindexed_gdf,
                                        first_test_timestep, last_test_timestep,
                                        num_geoids, bpr_uncertainty_samples=1)

In [14]:
print(f"2019 Average: {np.mean(bpr_over_time_zeroes[0])}")

bpr_samples_both_years = (np.array(bpr_over_time_zeroes[0]) + \
                          np.array(bpr_over_time_zeroes[1]))/2
                        
print(f"""Zeroes model (Mean, 95% CI): {np.mean(bpr_samples_both_years)*100:.1f},
      ({np.percentile(bpr_samples_both_years,2.5)*100:.1f}-
       {np.percentile(bpr_samples_both_years,97.5)*100:.1f})""")


2019 Average: 0.2598248847926267
Zeroes model (Mean, 95% CI): 25.7,
      (25.7-
       25.7)


In [15]:
bpr_over_time_last_time = models.last_time_model(multiindexed_gdf, first_test_timestep, last_test_timestep, num_geoids,
                     1,bpr_uncertainty_samples=1,)

In [16]:
print(f"2019 Average: {np.mean(bpr_over_time_last_time[0])}")

bpr_samples_both_years = (np.array(bpr_over_time_last_time[0]) + \
                          np.array(bpr_over_time_last_time[1]))/2
                        
print(f"""Zeroes model (Mean, 95% CI): {np.mean(bpr_samples_both_years)*100:.1f},
      ({np.percentile(bpr_samples_both_years,2.5)*100:.1f}-
       {np.percentile(bpr_samples_both_years,97.5)*100:.1f})""")


2019 Average: 0.5595506912442397
Zeroes model (Mean, 95% CI): 54.0,
      (54.0-
       54.0)


In [17]:
bpr_over_time_avg_time = models.historical_average_model(multiindexed_gdf, first_test_timestep, last_test_timestep, num_geoids,
                     1, 7, bpr_uncertainty_samples=1,)

In [18]:
print(f"2019 Average: {np.mean(bpr_over_time_avg_time[0])}")

bpr_samples_both_years = (np.array(bpr_over_time_avg_time[0]) + \
                          np.array(bpr_over_time_avg_time[1]))/2
                        
print(f"""Zeroes model (Mean, 95% CI): {np.mean(bpr_samples_both_years)*100:.1f},
      ({np.percentile(bpr_samples_both_years,2.5)*100:.1f}-
       {np.percentile(bpr_samples_both_years,97.5)*100:.1f})""")


2019 Average: 0.6351958525345621
Zeroes model (Mean, 95% CI): 62.1,
      (62.1-
       62.1)


Make Scikit models

In [19]:
# Identical models, features are only difference
linear_poisson_weighted_avg = sklearn.linear_model.PoissonRegressor()
linear_poisson = sklearn.linear_model.PoissonRegressor()

# Params selected via grid search on validation. Need to re-do grid search for chicago
hist_poisson =   HistGradientBoostingRegressor(loss="poisson", max_iter=10000, max_depth=3, max_leaf_nodes=2,
                                               l2_regularization=1, min_samples_leaf=100 )

In [20]:
bpr_over_time_weight_avg = models.scikit_model(multiindexed_gdf, x_BSF_death_only,
                                               y_BS_death_only, x_test_BSF_death_only,
                                               linear_poisson_weighted_avg,
                                               first_test_timestep, last_test_timestep,
                                               bpr_uncertainty_samples=5)

In [21]:
print(f"2019 Average: {np.mean(bpr_over_time_weight_avg[0])}")

bpr_samples_both_years = (np.array(bpr_over_time_weight_avg[0]) + \
                          np.array(bpr_over_time_weight_avg[1]))/2
                        
print(f"""Zeroes model (Mean, 95% CI): {np.mean(bpr_samples_both_years)*100:.1f},
      ({np.percentile(bpr_samples_both_years,2.5)*100:.1f}-
       {np.percentile(bpr_samples_both_years,97.5)*100:.1f})""")


2019 Average: 0.6203416933002156
Zeroes model (Mean, 95% CI): 58.9,
      (58.1-
       59.5)


In [22]:
bpr_over_time_linear = models.scikit_model(multiindexed_gdf, x_BSF,
                                               y_BS, x_test_BSF,
                                               linear_poisson,
                                               first_test_timestep, last_test_timestep,
                                               bpr_uncertainty_samples=5)

/Users/jyontika/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_glm/glm.py:282: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res)


In [23]:
print(f"2019 Average: {np.mean(bpr_over_time_linear[0])}")

bpr_samples_both_years = (np.array(bpr_over_time_linear[0]) + \
                          np.array(bpr_over_time_linear[1]))/2
                        
print(f"""Zeroes model (Mean, 95% CI): {np.mean(bpr_samples_both_years)*100:.1f},
      ({np.percentile(bpr_samples_both_years,2.5)*100:.1f}-
       {np.percentile(bpr_samples_both_years,97.5)*100:.1f})""")


2019 Average: 0.6333870071125552
Zeroes model (Mean, 95% CI): 61.2,
      (60.9-
       61.8)


In [24]:
bpr_over_time_tree = models.scikit_model(multiindexed_gdf, x_BSF,
                                               y_BS, x_test_BSF,
                                               hist_poisson,
                                               first_test_timestep, last_test_timestep,
                                               bpr_uncertainty_samples=5)

In [25]:
print(f"2019 Average: {np.mean(bpr_over_time_tree[0])}")

bpr_samples_both_years = (np.array(bpr_over_time_tree[0]) + \
                          np.array(bpr_over_time_tree[1]))/2
                        
print(f"""Zeroes model (Mean, 95% CI): {np.mean(bpr_samples_both_years)*100:.1f},
      ({np.percentile(bpr_samples_both_years,2.5)*100:.1f}-
       {np.percentile(bpr_samples_both_years,97.5)*100:.1f})""")


2019 Average: 0.5575402367685763
Zeroes model (Mean, 95% CI): 55.5,
      (53.8-
       57.1)
